# **Mildew Detector for Cherry Trees**

## Objectives

* Fetch data from Kaggle and prepare it for further ML processing.

## Inputs

* Write here which data or information you need to run the notebook 
* Kaggle.json file -authentication token

## Outputs

* Write here which files, code or artefacts you generate by the end of the notebook 
* Generate Dataset: inputs/datasets/cherry_leaves

## Additional Comments

* In case you have any additional comments that don't fit in the previous bullets, please state them here. 



---

# Change working directory

* We are assuming you will store the notebooks in a subfolder, therefore when running the notebook in the editor, you will need to change the working directory

We need to change the working directory from its current folder to its parent folder
* We access the current directory with os.getcwd()

In [76]:
import numpy
import os
from pathlib import Path


Change the working directory

In [77]:
current_dir = os.getcwd()
current_dir

'/workspace/P5-Mildew-Detection-in-Cherry-Leaves'

We want to make the parent of the current directory the new current directory
* os.path.dirname() gets the parent directory
* os.chir() defines the new current directory

In [78]:
os.chdir('/workspace/P5-Mildew-Detection-in-Cherry-Leaves')
print("You set a new current directory")

You set a new current directory


Confirm the new current directory

In [79]:
current_dir = os.getcwd()
current_dir

'/workspace/P5-Mildew-Detection-in-Cherry-Leaves'

# Install Kaggle

Section 1 content

In [80]:
# Install Kaggle package
!pip install kaggle 

---

Change the Kaggle configuration directory to current working directory and permission of Kaggle authentication json.

In [81]:
os.environ['KAGGLE_CONFIG_DIR'] = os.getcwd()
! chmod 600 kaggle.json

Set Kaggle Dataset and download it.

In [ ]:
KaggleDatasetPath = "codeinstitute/cherry-leaves"
DestinationFolder = "inputs/cherry_leaves"
! kaggle datasets download -d {KaggleDatasetPath} -p {DestinationFolder}


Unzip the downloaded file, and delete the zip file.

In [ ]:
import zipfile
with zipfile.ZipFile(DestinationFolder + '/cherry-leaves.zip', 'r') as zip_ref:
    zip_ref.extractall(DestinationFolder)

os.remove(DestinationFolder + '/cherry-leaves.zip')

# Data Preparation

Check and Remove non-image files

---

In [87]:

def remove_non_image_file(my_data_dir):
    image_extension = ('.png', '.jpg', '.jpeg')
    folders = os.listdir(my_data_dir)
    for folder in folders:
        files = os.listdir(my_data_dir + '/' + folder)
        # print(files)
        i = []
        j = []
        for given_file in files:
            if not given_file.lower().endswith(image_extension):
                file_location = my_data_dir + '/' + folder + '/' + given_file
                os.remove(file_location)  # remove non image file
                i.append(1)
            else:
                j.append(1)
                pass
        print(f"Folder: {folder} - has image file", len(j))
        print(f"Folder: {folder} - has non-image file", len(i))

In [89]:
remove_non_image_file(my_data_dir='inputs/cherry_leaves/cherry-leaves')

Folder: healthy - has image file 2104
Folder: healthy - has non-image file 0
Folder: powdery_mildew - has image file 2104
Folder: powdery_mildew - has non-image file 0


---

# Push files to Repo

* If you don't need to push files to Repo, you may replace this section with "Conclusions and Next Steps" and state your conclusions and next steps.

In [ ]:
import os
try:
    # create here your folder
    # os.makedirs(name='')
except Exception as e:
    print(e)
